In [1]:
import yfinance as yf

sp400 = yf.download('^SP400')
sp600 = yf.download('^SP600')

tickers_sp1000 = sp400.index.tolist() + sp600.index.tolist()

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [2]:
import pandas as pd

class WikipediaStockListing:
    def __init__(self, market):
        self.market = market
    
    def read(self, bucket : str):
        url = f'https://en.wikipedia.org/wiki/List_of_S&P_{bucket}_companies'
        df = pd.read_html(url, header=0)[0]
        if bucket == 500 or bucket == 400:
            cols_ren = {'Security':'Name', 'Ticker symbol':'Symbol', 'GICS Sector':'Sector', 'GICS Sub-Industry':'Industry'}
        elif bucket == 600:
            cols_ren = {'Company':'Name', 'Ticker symbol':'Symbol', 'GICS Sector':'Sector', 'GICS Sub-Industry':'Industry'}
        df = df.rename(columns = cols_ren)
        df = df[['Symbol', 'Name', 'Sector', 'Industry']]
        df['Symbol'] = df['Symbol'].str.replace('.', '', regex=False)
        return df

In [3]:
sp400_list = WikipediaStockListing('S&P').read(400)

In [4]:
sp500_list = WikipediaStockListing('S&P').read(500)

In [5]:
sp600_list = WikipediaStockListing('S&P').read(600)

In [16]:
sp600_list.loc[sp600_list[sp600_list['Symbol'] == 'CWENA'].index, 'Symbol'] = 'CWEN-A'

In [19]:
sp400_list.loc[sp400_list[sp400_list['Symbol'] == 'PNM'].index, 'Symbol'] = 'TXNM'

In [22]:
sp600_list.loc[sp600_list[sp600_list['Symbol'] == 'MOGA'].index, 'Symbol'] = 'MOG-A'

In [23]:
sp500_list.loc[sp500_list[sp500_list['Symbol'] == 'BFB'].index, 'Symbol'] = 'BF-B'

In [24]:
sp500_list.loc[sp500_list[sp500_list['Symbol'] == 'BRKB'].index, 'Symbol'] = 'BRK-B'

In [30]:
sp600_list.loc[sp600_list[sp600_list['Symbol'] == 'UCBI'].index, 'Symbol'] = 'UCB'

In [31]:
from sqlalchemy import create_engine

engine = create_engine(
    'sqlite:///S&P_company_list.db', 
    echo = False
)

In [32]:
sp400_list.to_sql(
    'sp400_ticker_list',
    con = engine, 
    if_exists = 'replace'
)
sp500_list.to_sql(
    'sp500_ticker_list',
    con = engine, 
    if_exists = 'replace'
)
sp600_list.to_sql(
    'sp600_ticker_list',
    con = engine, 
    if_exists = 'replace'
)

602

### Stock Data Download

In [33]:
spx = sp500_list['Symbol'].tolist() + sp600_list['Symbol'].tolist() + sp400_list['Symbol'].tolist()

In [34]:
prices = yf.download(
    spx,
    start = '1990-01-01'
)

[*********************100%%**********************]  1506 of 1506 completed


In [35]:
prices.to_parquet(
    '../Data/S&P_stocks_prices.parquet'
)